# Dataset

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

# TODO: Fill this according to your drive loc
# DAVID: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
# STANLEY: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/'
# STANLEY 2 :
# DEEPAK: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['data', 'requirements.txt', 'utils', 'params.json', 'reconstruction.py', 'hello_testing.py', '__pycache__', 'PCN.ipynb', 'Reconstruct.ipynb', 'Grasp Generation.ipynb', 'dataset.py', '.ipynb_checkpoints', 'config_files', 'model', 'results', 'scripts', 'obj_paths.pkl', 'Dataset.ipynb', 'eval']


If you did everything right, you should see something like this:
```
['DeepSDF.ipynb', 'data', 'models', 'requirements.txt', 'utils', 'params.json', 'Reconstruct.ipynb', 'Dataset.ipynb', 'hello_testing.py', 'model.py', '__pycache__']
```

<font color='red'>NOTE:</font> You don't have to clone the shared drive.
1. Create a shortcut of the shared folder (DeepSDF) and place in your drive.
2. Then change the directory on the notebook according to your gdrive.

In [3]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

from hello_testing import hello
hello()

dataset_path = os.path.join(GOOGLE_DRIVE_PATH, 'dataset.py')
dataset_edit_time = time.ctime(os.path.getmtime(dataset_path))
print('dataset.py last edited on %s' % dataset_edit_time)

Working!
dataset.py last edited on Tue Nov 21 21:22:54 2023


Navigate to :

```drive > .... > 3D-RP-Project > dataset.py```

Double-click on the file to open it on the side.

## ShapeNet Dataloader

## dataset.py

In [4]:
!pip3 install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 8.2 MB/s eta 0:00:00


In [5]:
!pip uninstall torch -y
!pip install torch==2.0.1

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Found existing installation: torch 2.1.0+cu118
Uninstalling torch-2.1.0+cu118:
  Successfully uninstalled torch-2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 

In [6]:
!pip install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 5.4 MB/s eta 0:00:00


In [7]:
!pip install point-cloud-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 7.0 MB/s eta 0:00:00


In [8]:
import numpy as np
import results
import os
from utils import utils_mesh
import point_cloud_utils as pcu
import data.ShapeNetCoreV2 as ShapeNetCoreV2
from glob import glob
from datetime import datetime
import config_files
import yaml
import pybullet as pb
import trimesh
"""
For each object, sample points and store their distance to the nearest triangle.
Sampling follows the approach used in the DeepSDF paper.
"""

def combine_sample_latent(samples, latent_class):
    """Combine each sample (x, y, z) with the latent code generated for this object.
    Args:
        samples: collected points, np.array of shape (N, 3)
        latent: randomly generated latent code, np.array of shape (1, args.latent_size)
    Returns:
        combined hstacked latent code and samples, np.array of shape (N, args.latent_size + 3)
    """
    latent_class_full = np.tile(latent_class, (samples.shape[0], 1))   # repeat the latent code N times for stacking
    return np.hstack((latent_class_full, samples))


##### Time consuming so run once

In [ ]:
cfg_path = os.path.join(os.path.dirname(config_files.__file__), 'extract_sdf.yaml')
with open(cfg_path, 'rb') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
# Full paths to all .obj
obj_paths = glob(os.path.join(os.path.dirname(ShapeNetCoreV2.__file__), '*', '*', 'models', '*.obj'))


In [ ]:
len(obj_paths)

2702

In [ ]:
# import pickle
# # Save the list to a file
# with open('obj_paths.pkl', 'wb') as f:
#     pickle.dump(obj_paths, f)



In [ ]:
# Load the list back from the file
# import pickle
# with open('obj_paths.pkl', 'rb') as f:
#     obj_paths = pickle.load(f)

FileNotFoundError: ignored

In [ ]:
# len(obj_paths)

In [ ]:
# [can, mug, bowl, camera]
target_folders = ['02942699'] # camera
# target_folders = ['02946921'] #can
# target_folders = ['02880940'] # bowl
# target_folders = ['03797390'] # mug
# target_folders = ['02691156'] # airplane
# Filter obj_paths to include only paths in target_folders
obj_paths = [path for path in obj_paths if any(folder in path for folder in target_folders)]

In [ ]:
len(obj_paths)

113

In [ ]:
obj_paths

['/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/1a54a2319e87bd4071d03b466c72ce41/models/model_normalized.obj',
 '/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/1a6ad7a24bb89733f412783097373bdc/models/model_normalized.obj',
 '/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/1a04e3eab45ca15dd86060f189eb133/models/model_normalized.obj',
 '/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/3a72adcf14ccec9965f50257ecdfa5c7/models/model_normalized.obj',
 '/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/3a3403fb14bdb46823e79e99b949a341/models/model_normalized.obj',
 '/content/drive/My Drive/ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/data/ShapeNetCoreV2/02691156/3a7e89dc355faac111f5789deac2dcac/models/model_normalized

## **TODO:** FOR DAVID dictionary
## **I. Combining existing parts**
1. Load obj_paths.pkl file
- You will have to upload file to colab by first downloading from `DeepSDF/obj_paths.pkl`
2. First attempt to combine the dictionaries:
- 3 different dicts with 3 parts each under results/to combine/
- `idx_str2int_dict_{i}_.npy` with i = {325,675} and `last_idx_str2int_dict.npy`
- `idx_int2str_dict_{i}_.npy` with i = {325,675} and `last_idx_int2str_dict.npy`
- `samples_dict_{i}_ShapeNetCoreV2.npy` with i = {325,675} and `last_samples_dict__ShapeNetCoreV2.npy`

**<font color='red'>NOTE:</font>**
Don't run


In [ ]:
len(obj_paths)

200

## **ONLY FOR COLLECTING SDFs and LATENT CLASS CODE**
### <font color='red'>NOTE:</font> Don't run this for combining, use only for collecting sdfs for individual objects

###  **II. INDIVIDUAL DICTIONARIES FOR EACH OBJECT TYPE**
1. Load obj_paths.pkl file
- You will have to upload file to colab by first downloading from `DeepSDF/obj_paths.pkl`
- <font color='red'>NOTE:</font> Size of obj_paths.pkl file will be 1006
- 25 objects of bottle is skipped as it is either corrupt or causes issue : This is handled in the try-except snippet so don't worry
2. Set object_name accordingly
3. Use indices below (Copy paste this exact):
- Bottle (2876657): `obj_paths[0:473]`
- Can (2946921): `obj_paths[473:581]`
- Mug (3797390): `obj_paths[581:795]`
- Bowl (2880940): `obj_paths[795:981]`
- Camera (): `[982:]`
4. Make sure to `i = 0` at the start, don't need to change this


In [ ]:
#object_name = 'camera'
# object_name = 'can'
# object_name = 'bowl'
# object_name = 'mug'
object_name = 'airplane'

TODO: David: For report purposes, please record the time it took for collecting sdf samples for each object category

Use python time() library

In [ ]:
import time

# File to store the samples and SDFs
samples_dict = dict()
start = time.time()
# Store conversion between object index (int) and its folder name (str)
idx_str2int_dict = dict()
idx_int2str_dict = dict()
i=0 # TODO: Use stanley's indices
for obj_idx, obj_path in enumerate(obj_paths): # TODO: Indices for individual objects: Example bottle

    # Object unique index. Str to int by byte encoding
    obj_idx_str = os.sep.join(obj_path.split(os.sep)[-4:-2]) # e.g. '02958343/1a2b3c4d5e6f7g8h9i0j'
    idx_str2int_dict[obj_idx_str] = obj_idx
    idx_int2str_dict[obj_idx] = obj_idx_str

    # Dictionary to store the samples and SDFs
    samples_dict[obj_idx] = dict()

    try:
        verts, faces = pcu.load_mesh_vf(obj_path)

        # Convert to watertight mesh
        mesh_original = utils_mesh._as_mesh(trimesh.load(obj_path))

        if not mesh_original.is_watertight:
            verts, faces = pcu.make_mesh_watertight(mesh_original.vertices, mesh_original.faces, 50000)
            mesh_original = trimesh.Trimesh(vertices=verts, faces=faces)

    except Exception as e:
        print(e)
        continue

    # In Shapenet, the front is the -Z axis with +Y still being the up axis. Rotate objects to align with the canonical axis.
    mesh = utils_mesh.shapenet_rotate(mesh_original)
    verts = np.array(mesh.vertices)

    # Generate random points in the predefined volume that surrounds all the shapes.
    # NOTE: ShapeNet shapes are normalized within [-1, 1]^3
    # p_vol = np.random.rand(cfg['num_samples_in_volume'], 3) * 2 - 1
    p_vol = np.random.rand(3000, 3) * 2 - 1
    # Sample within the object's bounding box. This ensures a higher ratio between points inside and outside the surface.
    v_min, v_max = verts.min(0), verts.max(0)
    #p_bbox = np.random.uniform(low=[v_min[0], v_min[1], v_min[2]], high=[v_max[0], v_max[1], v_max[2]], size=(cfg['num_samples_in_bbox'], 3))
    p_bbox = np.random.uniform(low=[v_min[0], v_min[1], v_min[2]], high=[v_max[0], v_max[1], v_max[2]], size=(10000, 3))
    # Sample points on the surface as face ids and barycentric coordinates
    #fid_surf, bc_surf = pcu.sample_mesh_random(verts, faces, cfg['num_samples_on_surface'])
    fid_surf, bc_surf = pcu.sample_mesh_random(verts, faces, 10000)

    try:
      # Compute 3D coordinates and normals of surface samples
      p_surf = pcu.interpolate_barycentric_coords(faces, fid_surf, bc_surf, verts)
      if p_surf is None:

        # Skip the iteration if an IndexError occurred
        continue
    except Exception as e:
        print(e)
        continue
    p_total = np.vstack((p_vol, p_bbox, p_surf))

    # Comput the SDF of the random points
    sdf, _, _  = pcu.signed_distance_to_mesh(p_total, verts, faces)

    samples_dict[obj_idx]['sdf'] = sdf
    # The samples are p_total, while the latent class is [obj_idx]
    samples_dict[obj_idx]['samples_latent_class'] = combine_sample_latent(p_total, np.array([obj_idx], dtype=np.int32))
    i+=1
    if i%25==0:
      print("iter: ", i)
      # np.save(os.path.join(os.path.dirname(results.__file__), f'samples_dict_{i}_ShapeNetCoreV2.npy'), samples_dict)
      # np.save(os.path.join(os.path.dirname(results.__file__), f'idx_str2int_dict_{i}_.npy'), idx_str2int_dict)
      # np.save(os.path.join(os.path.dirname(results.__file__), f'idx_int2str_dict_{i}_.npy'), idx_int2str_dict)


end = time.time()

print(f"Time to load {object_name}: {end - start} seconds")

np.save(os.path.join(os.path.dirname(results.__file__), f'{object_name}/samples_dict_ShapeNetCore.npy'), samples_dict) # TODO: Include object_name at start

np.save(os.path.join(os.path.dirname(results.__file__), f'{object_name}/idx_str2int_dict.npy'), idx_str2int_dict)
np.save(os.path.join(os.path.dirname(results.__file__), f'{object_name}/idx_int2str_dict.npy'), idx_int2str_dict)

NameError: ignored

In [ ]:
len(samples_dict)

NameError: ignored

## Load parts and save

### <font color='red'>NOTE:</font> Run these parts below before BACKUP section to load the parts and combine. You'll have to debug this part and sort the 980 and 981 issue.

1. Check:
- Size of each dictionary
- Size of each dictionary part
- If all consistent, combining logic must be wrong

In [ ]:
import numpy as np
import os
# Load the data from the .npy file
dict1 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/samples_dict_325_ShapeNetCoreV2.npy'), allow_pickle=True).item()
dict2 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/samples_dict_675_ShapeNetCoreV2.npy'), allow_pickle=True).item()
dict3 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/last_samples_dict_ShapeNetCore.npy'), allow_pickle=True).item()

# Combine dictionaries and adjust keys
combined_dict = {}

# print(f"Size of 1: {len(dict1)}")
# print(f"Size of 2: {len(dict2)}")
# print(f"Size of 3: {len(dict3)}")

count = 0
for i, d in enumerate([dict1, dict2, dict3]):
    # print(i)
    # count = 0
    for key, value in d.items():
        # count2 = i * len(d) + key
        # if count == count2:
        #   print("key")
        # else:
        #   print("fekkkkkkk")
        #   print(count)
        #   print(count2)
        # count = i * len(d) + key
        # print(i * len(d) + key)
        combined_dict[count] = value
        count += 1

np.save(os.path.join(GOOGLE_DRIVE_PATH,f'results/samples_dict_ShapeNetCore.npy'), combined_dict)

In [ ]:
print(len(combined_dict.keys()))

981


In [ ]:
# Load the data from the .npy file
dict1 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/idx_int2str_dict_325_.npy'), allow_pickle=True).item()
dict2 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/idx_int2str_dict_675_.npy'), allow_pickle=True).item()
dict3 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/last_idx_int2str_dict.npy'), allow_pickle=True).item()
# Combine dictionaries and adjust keys
combined_dict = {}

n = 0
for i, d in enumerate([dict1, dict2, dict3]):
    for key, value in d.items():
        combined_dict[n] = value
        n += 1
        # combined_dict[i * len(d) + key] = value

np.save(os.path.join(GOOGLE_DRIVE_PATH,f'results/idx_int2str_dict.npy'), combined_dict)

In [ ]:
# Load the data from the .npy file
dict1 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/idx_str2int_dict_325_.npy'), allow_pickle=True).item()
dict2 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/idx_str2int_dict_675_.npy'), allow_pickle=True).item()
dict3 = np.load(os.path.join(GOOGLE_DRIVE_PATH,f'results/to combine/last_idx_str2int_dict.npy'), allow_pickle=True).item()

# Concatenate dictionaries without adjusting keys
combined_dict = dict1.copy()
combined_dict.update(dict2)
combined_dict.update(dict3)

np.save(os.path.join(GOOGLE_DRIVE_PATH,f'results/idx_str2int_dict.npy'), combined_dict)

In [ ]:
print(len(combined_dict.keys()))


981


[326,325,330]

In [ ]:
print("Lengths: {}, {}, {}".format(len(first),len(second),len(third)))

NameError: ignored

# BACKUP

In [ ]:
def main(cfg):

    # Full paths to all .obj
    obj_paths = glob(os.path.join(os.path.dirname(ShapeNetCoreV2.__file__), '*', '*', 'models', '*.obj'))

    # File to store the samples and SDFs
    samples_dict = dict()

    # Store conversion between object index (int) and its folder name (str)
    idx_str2int_dict = dict()
    idx_int2str_dict = dict()
    i=0
    for obj_idx, obj_path in enumerate(obj_paths):

        # Object unique index. Str to int by byte encoding
        obj_idx_str = os.sep.join(obj_path.split(os.sep)[-4:-2]) # e.g. '02958343/1a2b3c4d5e6f7g8h9i0j'
        idx_str2int_dict[obj_idx_str] = obj_idx
        idx_int2str_dict[obj_idx] = obj_idx_str

        # Dictionary to store the samples and SDFs
        samples_dict[obj_idx] = dict()

        try:
            verts, faces = pcu.load_mesh_vf(obj_path)

            # Convert to watertight mesh
            mesh_original = utils_mesh._as_mesh(trimesh.load(obj_path))

            if not mesh_original.is_watertight:
                verts, faces = pcu.make_mesh_watertight(mesh_original.vertices, mesh_original.faces, 50000)
                mesh_original = trimesh.Trimesh(vertices=verts, faces=faces)

        except Exception as e:
            print(e)
            continue

        # In Shapenet, the front is the -Z axis with +Y still being the up axis. Rotate objects to align with the canonical axis.
        mesh = utils_mesh.shapenet_rotate(mesh_original)
        verts = np.array(mesh.vertices)

        # Generate random points in the predefined volume that surrounds all the shapes.
        # NOTE: ShapeNet shapes are normalized within [-1, 1]^3
        # p_vol = np.random.rand(cfg['num_samples_in_volume'], 3) * 2 - 1
        p_vol = np.random.rand(3000, 3) * 2 - 1
        # Sample within the object's bounding box. This ensures a higher ratio between points inside and outside the surface.
        v_min, v_max = verts.min(0), verts.max(0)
        #p_bbox = np.random.uniform(low=[v_min[0], v_min[1], v_min[2]], high=[v_max[0], v_max[1], v_max[2]], size=(cfg['num_samples_in_bbox'], 3))
        p_bbox = np.random.uniform(low=[v_min[0], v_min[1], v_min[2]], high=[v_max[0], v_max[1], v_max[2]], size=(10000, 3))
        # Sample points on the surface as face ids and barycentric coordinates
        #fid_surf, bc_surf = pcu.sample_mesh_random(verts, faces, cfg['num_samples_on_surface'])
        fid_surf, bc_surf = pcu.sample_mesh_random(verts, faces, 10000)

        try:
          # Compute 3D coordinates and normals of surface samples
          p_surf = pcu.interpolate_barycentric_coords(faces, fid_surf, bc_surf, verts)
          if p_surf is None:

            # Skip the iteration if an IndexError occurred
            continue
        except Exception as e:
            print(e)
            continue
        p_total = np.vstack((p_vol, p_bbox, p_surf))

        # Comput the SDF of the random points
        sdf, _, _  = pcu.signed_distance_to_mesh(p_total, verts, faces)

        samples_dict[obj_idx]['sdf'] = sdf
        # The samples are p_total, while the latent class is [obj_idx]
        samples_dict[obj_idx]['samples_latent_class'] = combine_sample_latent(p_total, np.array([obj_idx], dtype=np.int32))
        i+=1
        if i%1000==0:
          print("iter: ", i)
          np.save(os.path.join(os.path.dirname(results.__file__), f'samples_dict_{i}_{cfg["dataset"]}.npy'), samples_dict)
          np.save(os.path.join(os.path.dirname(results.__file__), f'idx_str2int_dict_{i}_.npy'), idx_str2int_dict)
          np.save(os.path.join(os.path.dirname(results.__file__), f'idx_int2str_dict_{i}_.npy'), idx_int2str_dict)

    np.save(os.path.join(os.path.dirname(results.__file__), f'samples_dict_{cfg["dataset"]}.npy'), samples_dict)

    np.save(os.path.join(os.path.dirname(results.__file__), f'idx_str2int_dict.npy'), idx_str2int_dict)
    np.save(os.path.join(os.path.dirname(results.__file__), f'idx_int2str_dict.npy'), idx_int2str_dict)


if __name__=='__main__':
    cfg_path = os.path.join(os.path.dirname(config_files.__file__), 'extract_sdf.yaml')
    with open(cfg_path, 'rb') as f:
        cfg = yaml.load(f, Loader=yaml.FullLoader)

    main(cfg)